In [1]:
%matplotlib inline

import numpy as np
# from numba.pycc import CC
from numba import njit, prange
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix, csr_matrix, bsr_matrix

def show_sparse_image(image, title):
    plt.spy(image)
    plt.title(title)
    plt.show()

# cc = CC('my_numba_test')

In [2]:
import random
# Test using numba parallel sum over rows
# @cc.export('do_numba_sum', 'f4(f4[:,:])')
# @njit( parallel = True )
# def do_numba_sum(vals):
#     n = vals.shape[0]
#     partials = np.zeros(n)
#     for p in prange(n):
#         partials[p] = np.sum(vals[p])
#     return np.sum(partials)

@njit( parallel = True)
def populate_buffers(l, kernel, workers):
    
    # Kernel info
    k = kernel.shape[0]
    k_h = (k - 1) // 2
    kernel = kernel.flatten()
    
    # How many elements per array
    m = l**2 * k**2
    print(f"m: {m}")
    # Buffer allocation
    row = np.zeros(m, dtype=np.intc)
    col = np.zeros(m, dtype=np.intc)
    val = np.zeros(m, dtype=np.float32)
    
    # Work distribution
    work_size = m // workers
    print(f"Work size: {work_size}")

    # Populating buffers using strided workers
    for t in prange(workers):
        row[t*work_size : (t+1)*work_size] = random.randint(0, l**2)
        col[t*work_size : (t+1)*work_size] = random.randint(0, l**2)
        val[t*work_size : (t+1)*work_size] = random.randint(0, np.finfo(np.float32).max)
    
    return row, col, val

In [ ]:
%%time

def gkern(l=5, sig=1.):
    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

l = 100
k = 9
gauss = gkern(k).flatten()

workers = 10
row, col, val = populate_buffers(l, gauss, workers)
print((row.size * row.itemsize) / 10**6)
print((col.size * col.itemsize) / 10**6)
print((val.size * val.itemsize) / 10**6)
matrix =  coo_matrix((val, (row, col)), shape=(l**2, l**2))
print((matrix.col.nbytes + matrix.row.nbytes + matrix.data.nbytes) / 10**6)
matrix = matrix.tocsr()
print((matrix.data.nbytes + matrix.indptr.nbytes + matrix.indices.nbytes) / 10**6)
show_sparse_image(matrix, "Bullsit")
print(matrix)

m: 65610000
Work size: 6561000


In [6]:
l = 5
k = 3
gauss = gkern(k).flatten()

rows = np.zeros(l**2 * k**2, dtype=np.float32)
cols = np.zeros(l**2 * k**2, dtype=np.float32)
data = np.zeros(l**2 * k**2, dtype=np.float32)

for r in range(l**2): # for each
    s = r*k**2
    e = s + k**2
    # print(f"Row {r}: start/end = {s}/{e}")
    rows[s:e] = r
    cols[s:e] = np.arange(k**2) + r
    data[s:e] = gauss

# print(rows)
# print(cols)
# print(data)
print((rows.size * rows.itemsize) / 10**6)
print((cols.size * cols.itemsize) / 10**6)
print((data.size * data.itemsize) / 10**6)
    
matrix = bsr_matrix((data, (rows, cols)), shape=(l**2, l**2 + k**2 - 1))
print((matrix.data.nbytes + matrix.indptr.nbytes + matrix.indices.nbytes) / 10**6)
# show_sparse_image(matrix, "Sparse Matrix")

# print(f"Matrix data: {matrix.data}")
print(f"Matrix indices: {matrix.indices}")
print(f"Matrix indptr: {matrix.indptr}")

0.0009
0.0009
0.0009
0.001904
Matrix indices: [ 0  1  2  3  4  5  6  7  8  1  2  3  4  5  6  7  8  9  2  3  4  5  6  7
  8  9 10  3  4  5  6  7  8  9 10 11  4  5  6  7  8  9 10 11 12  5  6  7
  8  9 10 11 12 13  6  7  8  9 10 11 12 13 14  7  8  9 10 11 12 13 14 15
  8  9 10 11 12 13 14 15 16  9 10 11 12 13 14 15 16 17 10 11 12 13 14 15
 16 17 18 11 12 13 14 15 16 17 18 19 12 13 14 15 16 17 18 19 20 13 14 15
 16 17 18 19 20 21 14 15 16 17 18 19 20 21 22 15 16 17 18 19 20 21 22 23
 16 17 18 19 20 21 22 23 24 17 18 19 20 21 22 23 24 25 18 19 20 21 22 23
 24 25 26 19 20 21 22 23 24 25 26 27 20 21 22 23 24 25 26 27 28 21 22 23
 24 25 26 27 28 29 22 23 24 25 26 27 28 29 30 23 24 25 26 27 28 29 30 31
 24 25 26 27 28 29 30 31 32]
Matrix indptr: [  0   9  18  27  36  45  54  63  72  81  90  99 108 117 126 135 144 153
 162 171 180 189 198 207 216 225]
